In [3]:
# import the used libraries
import numpy as np
import matplotlib.pyplot as plt
# import statsmodels.api as sm
import QuantLib as ql
%matplotlib inline

In [16]:
# Setting evaluation date
today = ql.Date(7,4,2015)
ql.Settings.instance().setEvaluationDate(today)

In [18]:
# Setup Marketdata
rate = ql.SimpleQuote(0.05)
rate_handle = ql.QuoteHandle(rate)
dc = ql.Actual365Fixed()
yts = ql.FlatForward(today, rate_handle, dc)
yts.enableExtrapolation()
hyts = ql.RelinkableYieldTermStructureHandle(yts)
t0_curve = ql.YieldTermStructureHandle(yts)
euribor6m = ql.Euribor6M(hyts)
cal = ql.TARGET()

In [20]:
# Setup a dummy portfolio with two Swaps
def makeSwap(start, maturity, nominal, fixedRate, index, typ=ql.VanillaSwap.Payer):
    """
    creates a plain vanilla swap with fixedLegTenor 1Y
    
    parameter:
        
        start (ql.Date) : Start Date
        
        maturity (ql.Period) : SwapTenor
        
        nominal (float) : Nominal
        
        fixedRate (float) : rate paid on fixed leg
        
        index (ql.IborIndex) : Index
        
    return: tuple(ql.Swap, list<Dates>) Swap and all fixing dates
    
        
    """
    end = ql.TARGET().advance(start, maturity)
    fixedLegTenor = ql.Period("1y")
    fixedLegBDC = ql.ModifiedFollowing
    fixedLegDC = ql.Thirty360(ql.Thirty360.BondBasis)
    spread = 0.0
    fixedSchedule = ql.Schedule(start,
                                end, 
                                fixedLegTenor, 
                                index.fixingCalendar(), 
                                fixedLegBDC,
                                fixedLegBDC, 
                                ql.DateGeneration.Backward,
                                False)
    floatSchedule = ql.Schedule(start,
                                end,
                                index.tenor(),
                                index.fixingCalendar(),
                                index.businessDayConvention(),
                                index.businessDayConvention(),
                                ql.DateGeneration.Backward,
                                False)
    swap = ql.VanillaSwap(typ, 
                          nominal,
                          fixedSchedule,
                          fixedRate,
                          fixedLegDC,
                          floatSchedule,
                          index,
                          spread,
                          index.dayCounter())
    return swap, [index.fixingDate(x) for x in floatSchedule][:-1]

In [19]:
def makeSwaption(swap, callDates, settlement):
    """
    Creates the swaption of the underlying swap.
    
    If there is only one callDate in the list of call dates it will be a European swaption
    otherwise a Bermudan.
    """
    if len(callDates) == 1:
        exercise = ql.EuropeanExercise(callDates[0])
    else:
        exercise = ql.BermudanExercise(callDates)
    return ql.Swaption(swap, exercise, settlement)

In [21]:
settlementDate = today + ql.Period("2D")

swaps = [makeSwap(settlementDate,
                  ql.Period("5Y"),
                  1e6,
                  0.03,
                  euribor6m)
        ]

calldates = [ql.Date(7,4,2016), ql.Date(6,4,2017), ql.Date(5,4, 2018), ql.Date(5,4,2019)]
 
swaptions = [makeSwaption(swap, 
                          calldates, 
                          ql.Settlement.Physical) 
             for swap, fd in swaps]

calldates = [euribor6m.valueDate(d) for d in calldates]

In [22]:
#%%timeit
# Setup pricing engine and calculate the npv of the underlying swap
engine = ql.DiscountingSwapEngine(hyts)
for swap, fixingDates in swaps:
    swap.setPricingEngine(engine)
    print("Swap NPV at time 0: %.4f" % swap.NPV())

Swap NPV at time 0: 91979.2517


In [8]:
floSchedule = np.arange(0,10.5,0.5)
fixShcedule = np.arange(0,11,1)
initRate    = 0.04
b           = 0.0125
beta        = 0.25
sigma       = 0.02

In [9]:
model = ql.Vasicek(r0=initRate, a=beta, b=beta*b, sigma = sigma)

In [14]:
ql.VanillaSwap.Payer

1

In [15]:
ql.Gaussian1dSwaptionEngine(model)

TypeError: Wrong number or type of arguments for overloaded function 'new_Gaussian1dSwaptionEngine'.
  Possible C/C++ prototypes are:
    Gaussian1dSwaptionEngine::Gaussian1dSwaptionEngine(ext::shared_ptr< Gaussian1dModel > const &,int const,Real const,bool const,bool const,Handle< YieldTermStructure > const &,Gaussian1dSwaptionEngine::Probabilities const)
    Gaussian1dSwaptionEngine::Gaussian1dSwaptionEngine(ext::shared_ptr< Gaussian1dModel > const &,int const,Real const,bool const,bool const,Handle< YieldTermStructure > const &)
    Gaussian1dSwaptionEngine::Gaussian1dSwaptionEngine(ext::shared_ptr< Gaussian1dModel > const &,int const,Real const,bool const,bool const)
    Gaussian1dSwaptionEngine::Gaussian1dSwaptionEngine(ext::shared_ptr< Gaussian1dModel > const &,int const,Real const,bool const)
    Gaussian1dSwaptionEngine::Gaussian1dSwaptionEngine(ext::shared_ptr< Gaussian1dModel > const &,int const,Real const)
    Gaussian1dSwaptionEngine::Gaussian1dSwaptionEngine(ext::shared_pt